In [1]:
from nepal.datasets import NYTimes, Vaccinations, CountyDistance

In [2]:
descriptor = CountyDistance(radius=100)
descriptor.collect()

In [3]:
data = descriptor.load()
data

,county1,mi_to_county,county2
0,01001,22.462994,01021
1,01001,26.844687,01085
2,01001,29.517586,01051
3,01001,30.776371,01047
4,01001,34.493443,01101
...,...,...,...
147151,72153,74.195923,72103
147152,72153,77.837814,72089
147153,72153,85.874054,72053
147154,72153,88.087677,72037


In [4]:
data.dtypes

county1          string
mi_to_county    Float32
county2          string
dtype: object